<a href="https://colab.research.google.com/github/m-sutt/nhanes_inferential_2023/blob/main/nhanes_inferential_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Assignment for HA507

# Data Files to load

#### demographic link: (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT)
- **Marital Status** (`DMDMARTZ`) - categorical, needs recoding (married or not married).
- **Education Level** (`DMDEDUC2`) - categorical, needs recoding (bachelor’s or higher vs. less than bachelor’s).
- **Age in Years** (`RIDAGEYR`) - continuous.

####  bp link: (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT)
- **Systolic Blood Pressure** (`BPXOSY3`) - continuous.
- **Diastolic Blood Pressure** (`BPXODI3`) - continuous.

#### vit d link: (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/VID_L.XPT)
- **Vitamin D Lab Interpretation** (`LBDVD2LC`) - categorical, two levels.

#### hep b link: (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HEPB_S_L.XPT)
- **Hepatitis B Lab Antibodies** (`LBXHBS`) - categorical, needs recoding to two levels.

#### kidney link (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/KIQ_U_L.XPT)
- **Weak/Failing Kidneys** (`KIQ022`) - categorical, can be treated as two levels.

#### sed link (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/PAQ_L.XPT)
- **Minutes of Sedentary Behavior** (`PAD680`) - continuous, needs cleaning (remove values `7777`, `9999`, and null).

#### self_weight link (https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/WHQ_L.XPT)
- **Current Self-Reported Weight** (`WHD020`) - continuous, needs cleaning (remove values `7777`, `9999`, and null).



In [4]:
import pandas as pd

# Load files demographics

demo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport') #marital status, ed level, age
bp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
lab_vitd = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/VID_L.XPT', format='xport')
lab_hepb = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HEPB_S_L.XPT', format='xport')
kidney = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/KIQ_U_L.XPT', format='xport')
sed = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/PAQ_L.XPT', format='xport')
self_weight = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/WHQ_L.XPT', format='xport')


In [5]:
# Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"
  ## Variables: `DMDMARTZ` (marital status) and `DMDEDUC2` (education level). Recode as specified.

# Data cleaning
## DMDMARTZ: 1 = marrried/living with partner 2 = divorced; 3 = never married
## DMDEDUC2: 1 = <9th grade; 2 = 9-11th; 3 = HS or GED; 4 = Some college or AA; 5 College grad; 9 = don't know


from scipy.stats import chi2_contingency


demo_stat = demo[['DMDMARTZ', 'DMDEDUC2']]
demo_stat['DMDMARTZ'] = demo_stat['DMDMARTZ'].astype(str)
demo_stat['DMDEDUC2'] = demo_stat['DMDEDUC2'].astype(str)
demo_stat.dtypes

# Select and clean relevant columns from demo data
demo_stat = demo[['DMDMARTZ', 'DMDEDUC2']].copy()

# Recode marital status: 1 = Married, 2 and 3 = Not Married
demo_stat['Marital_Status'] = demo_stat['DMDMARTZ'].replace({1: 'Married', 2: 'Not Married', 3: 'Not Married'})

# Recode education level: 5 = College Grad, 1-4 = Not College Grad, remove 'Don't know' coded as 9
demo_stat['Education_Level'] = demo_stat['DMDEDUC2'].replace({1: 'Not College Grad', 2: 'Not College Grad',
                                                              3: 'Not College Grad', 4: 'Not College Grad',
                                                              5: 'College Grad', 9: None})
# Drop rows with missing values
demo_stat = demo_stat.dropna(subset=['Marital_Status', 'Education_Level'])

# Create a contingency table for Marital Status and Education Level
contingency_table = pd.crosstab(demo_stat['Marital_Status'], demo_stat['Education_Level'])
print("Contingency Table:")
print(contingency_table)

# Perform Chi-Square test
stat, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"\nChi-Square Test Statistic: {stat}, p-value: {p_value}")
print("Expected Frequencies:")
print(expected)

# Interpretation
if p_value > 0.05:
    print("\nNo significant association between marital status and education.")
else:
    print("\nSignificant association between marital status and education.")

Contingency Table:
Education_Level  College Grad  Not College Grad
Marital_Status                                 
77.0                        2                 2
99.0                        2                 3
Married                  1631              2503
Not Married               990              2648

Chi-Square Test Statistic: 130.25908281829118, p-value: 4.755981297523769e-28
Expected Frequencies:
[[1.34944095e+00 2.65055905e+00]
 [1.68680118e+00 3.31319882e+00]
 [1.39464722e+03 2.73935278e+03]
 [1.22731654e+03 2.41068346e+03]]

Significant association between marital status and education.


<ipython-input-5-47e262d4c0ca>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_stat['DMDMARTZ'] = demo_stat['DMDMARTZ'].astype(str)
<ipython-input-5-47e262d4c0ca>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_stat['DMDEDUC2'] = demo_stat['DMDEDUC2'].astype(str)


In [6]:
# Question 2: "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"
     # Variables: `DMDMARTZ` (marital status, recoded) and `PAD680` (sedentary behavior time, cleaned).

# Merge demo and sed on SEQN to access DMDMARTZ (marital status) and PAD680 (sedentary behavior) in one DataFrame
merged_data = pd.merge(demo, sed[['SEQN', 'PAD680']], on='SEQN', how='inner')

# Filter relevant columns for analysis
demo_sedentary = merged_data[['DMDMARTZ', 'PAD680']].copy()

# Recode marital status: 1 = Married, 2 and 3 = Not Married
demo_sedentary['Marital_Status'] = demo_sedentary['DMDMARTZ'].replace({1: 'Married', 2: 'Not Married', 3: 'Not Married'})

# Clean PAD680: remove values 7777, 9999, and any nulls
demo_sedentary['Sedentary_Behavior'] = demo_sedentary['PAD680'].replace([7777, 9999], None)
demo_sedentary = demo_sedentary.dropna(subset=['Sedentary_Behavior', 'Marital_Status'])


# Perform T-test
from scipy.stats import ttest_ind

# separating data
married_sedentary = demo_sedentary[demo_sedentary['Marital_Status'] == 'Married']['Sedentary_Behavior']
not_married_sedentary = demo_sedentary[demo_sedentary['Marital_Status'] == 'Not Married']['Sedentary_Behavior']


# Perform the t-test
stat, p_value = ttest_ind(married_sedentary, not_married_sedentary, nan_policy='omit')
print(f"T-Test Statistic: {stat}, p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("No significant difference in mean sedentary behavior time between married and not married groups.")
else:
    print("Significant difference in mean sedentary behavior time between married and not married groups.")


T-Test Statistic: -3.8699896847970154, p-value: 0.00010973792037934772
Significant difference in mean sedentary behavior time between married and not married groups.


In [15]:
# Question 3: "How do age and marital status affect systolic blood pressure?"
     ## Variables: `RIDAGEYR` (age), `DMDMARTZ` (marital status, recoded), and `BPXOSY3` (systolic blood pressure).

import statsmodels.api as sm
import statsmodels.formula.api as ols

# Step 1: Extract columns
demo_data = demo[['SEQN', 'DMDMARTZ', 'RIDAGEYR']]
bp_data = bp[['SEQN', 'BPXOSY3']]

# Step 2: merge df (demo and bp data)on SEQN
merged_data = pd.merge(demo_data, bp_data, on='SEQN', how='left')

# Step 3: Clean and recode data
# Record martial status
merged_data['Marital_Status'] = merged_data['DMDMARTZ'].replace({1: 'Married', 2: 'Not Married', 3: 'Not Married'})

# Remove missing data
merged_data = merged_data.dropna(subset=['BPXOSY3', 'Marital_Status'])

# Step 4: Perform linear regression (create regression formula, fit model)
model = ols.ols('BPXOSY3 ~ RIDAGEYR + C(Marital_Status)', data=merged_data).fit()

# Step 5: Print results
print(model.summary())

### WORK IN PROGRESS ###
# Step 6: Interpretation
#X = bp_age_marital[['RIDAGEYR', 'Marital_Status']]
#X = pd.get_dummies(X, drop_first=True)  # Convert categorical variable to dummy/indicator variables
#X = sm.add_constant(X)  # Add an intercept term
#y = bp_age_marital['BPXOSY3']

#model = sm.OLS(y, X).fit()
#coefficients = model.params
#p_values = model.pvalues
#r_squared = model.rsquared

# Initialize interpretation list
#interpretation = []

# Age interpretation
#if p_values['RIDAGEYR'] < 0.05:
    #interpretation.append(f"Age is significantly associated with systolic blood pressure (coef: {coefficients['RIDAGEYR']:.4f}, p < 0.05).")
#else:
    #interpretation.append("Age is not significantly associated with systolic blood pressure.")

# Marital Status interpretation
#for var in X.columns[2:]:
 #   if p_values[var] < 0.05:
  #      interpretation.append(f"{var.replace('C(Marital_Status)[T.', '').replace(']', '')} has a significant effect on systolic blood pressure (coef: {coefficients[var]:.4f}, p < 0.05).")
   # else:
    #    interpretation.append(f"{var.replace('C(Marital_Status)[T.', '').replace(']', '')} does not have a significant effect on systolic blood pressure.")

# R-squared interpretation
#interpretation.append(f"The model explains {r_squared:.2%} of the variance in systolic blood pressure.")

# Display results
#for line in interpretation:
   # print(line)



                            OLS Regression Results                            
Dep. Variable:                BPXOSY3   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     227.3
Date:                Sat, 09 Nov 2024   Prob (F-statistic):          1.27e-181
Time:                        20:19:39   Log-Likelihood:                -24871.
No. Observations:                5842   AIC:                         4.975e+04
Df Residuals:                    5837   BIC:                         4.979e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

NameError: name 'bp_age_marital' is not defined

In [18]:
# Question 4**: "Is there a correlation between self-reported weight and minutes of sedentary behavior?"
     ## Variables: `WHD020` (self-reported weight, cleaned) and `PAD680` (sedentary behavior time, cleaned).


from scipy.stats import pearsonr, spearmanr

# Step 1: Extract columns and relevant data and merging data
weight_sedentary = self_weight[['SEQN', 'WHD020']].copy()
weight_sedentary = weight_sedentary.merge(sed[['SEQN', 'PAD680']], on='SEQN')

# Step 2: More cleanning.Drop missing values in columns
weight_sedentary.dropna(subset=['WHD020', 'PAD680'], inplace=True)

# Step 3: Perform correlation analysis
from scipy.stats import pearsonr

# Step 4: Perform Pearson correlation
corr, p_value = pearsonr(weight_sedentary['WHD020'], weight_sedentary['PAD680'])
print(f"Pearson Correlation Coefficient: {corr}, p-value: {p_value}")

# Step 5: Interpretation
if p_value < 0.05:
    print("There is a statistically significant correlation between self-reported weight and sedentary behavior time.")
else:
    print("There is no statistically significant correlation between self-reported weight and sedentary behavior time.")


Pearson Correlation Coefficient: 0.1046820984735302, p-value: 2.9649694069971987e-21
There is a statistically significant correlation between self-reported weight and sedentary behavior time.


In [20]:
# Question 5 (Creative Analysis)**: Develop your own unique question using at least one of the variables listed above. Ensure that your question can be answered using one of the following tests: chi-square, t-test, ANOVA, or correlation. Clearly state your question, explain why you chose the test, and document your findings.

# Question: Is there a correlation between age and minutes of sedentary behavior?

# Merge datasets on SEQN (sequence number)
merged_data = demo[['SEQN', 'RIDAGEYR']].merge(sed[['SEQN', 'PAD680']], on='SEQN', how='inner')

# Drop missing values
merged_data = merged_data.dropna(subset=['RIDAGEYR', 'PAD680'])

# Perform the correlation test
corr_stat, p_value = pearsonr(merged_data['RIDAGEYR'], merged_data['PAD680'])

# Print results
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}, p-value: {p_value:.4f}")

# Interpretation
if p_value > 0.05:
    print("No significant correlation between age and sedentary behavior time.")
else:
  print("There is a significant correlation between age and sedentary behavior time.")

Pearson Correlation Coefficient: 0.0414, p-value: 0.0002
There is a significant correlation between age and sedentary behavior time.
